# 🔍 ANÁLISE DE TÓPICOS COM BERTopic - INSTRUÇÕES

Este notebook realiza análise automática de tópicos em textos usando inteligência artificial.
Siga os passos abaixo para executar a análise:

1. INSTALAR E IMPORTAR (Execute esta célula primeiro)
2. CONFIGURAÇÕES (Enviar o seu arquivo CSV)
3. MODELAGEM DE TÓPICOS (Análise principal)
4. VISUALIZAÇÕES (Gráficos e resultados)
5. EXPORTAÇÃO (Salvar resultados)

Dica: Execute as células de cima pra baixo, na ordem, usando o botão ▶ ao lado de cada uma.

# 1️⃣ INSTALAR E IMPORTAR DEPENDÊNCIAS

A célula abaixo instala e importa todas as bibliotecas necessárias.

✏️ O que você precisa fazer:
- Apenas execute a célula (não precisa editar nada)
- Espere até ver a mensagem "Bibliotecas instaladas com sucesso!"

⏳ Tempo estimado: 1-2 minutos


In [ ]:
# 1. Instalações e Downloads
!pip install spacy bertopic wordcloud pandas
!python -m spacy download pt_core_news_sm
!python -m spacy download pt_core_news_md
!python -m spacy download pt_core_news_lg

# 1.1 Importações
import pandas as pd
import spacy
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from plotly.colors import qualitative

print("✅ Bibliotecas instaladas e importadas com sucesso!")

**📛 REINICIAR O KERNEL SE:**  
✔️ Células travando sem erro aparente  
✔️ Mensagens de "biblioteca não encontrada"  
✔️ Saídas inconsistentes entre execuções  

**🔧 COMO REINICIAR:**  
1. Menu: `Ambiente de execução` → `Reiniciar ambiente...`  
2. Ou use o atalho: `Ctrl + M .` (ponto)  
3. Execute todas as células novamente

# 2️⃣ CONFIGURAÇÕES

✏️ AJUSTE NA CÉLULA ABAIXO OS PARÂMETROS PRINCIPAIS:

1. Caminho do seu arquivo .CSV
(clique no ícone de pasta 📁 na coluna à esquerda, depois em 📤 upload,

   *   Clique no ícone de pasta 📁 na coluna à esquerda;
   *   Clique em 📤 upload;
   *   Selecione o .CSV do seu computador, aperte Open e OK;
   *   Clique com o botão direito no nome do arquivo e aperte em "Copiar Caminho";
   *   Cole esse caminho entre as aspas na célula abaixo.
2. Modelo de linguagem: pt_core_news_sm, pt_core_news_md e pt_core_news_lg, use md como padrão, sm para testes rápidos, lg apenas se precisar de alta precisão.
3. Palavras que deseja ignorar na análise (stopwords), é util para melhorar a qualidade dos tópicos identificados, remover ruído, otimizar o tempo de processamento.
4. Porcentagem de palavras muito frequentes que serão ignoradas.
4. Nome da coluna que contém os textos a analisar (evite nomear as colunas com duas palavras separadas por espaço).
5. Parâmetros avançados do modelo de tópicos, consulte a documentação em https://maartengr.github.io/BERTopic/api/bertopic.html (Ex.: min_topic_size: quanto menor o valor, mais tópicos são criados.)

🔧 Dicas:
- Altere apenas o que está entre aspas
- Para adicionar várias stopwords, separe por vírgulas, exemplo: ["palavra1", "palavra2", "palavra3"]


In [ ]:
# 2. #### CONFIGURÁVEIS ####

# 2.1 Defina o caminho para o seu CSV no Drive
CSV_PATH = '/content/nome-do-seu-arquivo.csv'  # ← ajuste para o seu caminho

# 2.2 Modelo spaCy para tokenização e POS-tagging. # Ex.: "pt_core_news_sm", "pt_core_news_md", "pt_core_news_lg", "en_core_web_sm", etc.
SPACY_MODEL = "pt_core_news_md"

# 2.3 Lista de palavras adicionais que você quer **forçar** a excluir # Exemplo: ["exemplo", "teste", "palavradescartar"]
CUSTOM_STOP_WORDS = ["exemplo", "palavradescartar", "teste", "outrapalavra"]

# 2.4 Descarta tokens que ocorrem em mais de 70% dos documentos
MAX_DOC_FREQ = 0.70

# 2.5 Nome da coluna do DataFrame que contém o texto a ser analisado. # Exemplo: "resenha", "descricao", "comentario"
TEXT_COLUMN = "nome_da_coluna"

# 2.6 Parâmetros do BERTopic:
#     language: para stop-words e pré-processamento (pode ser None para auto-detecção)
#     n_gram_range: (min, max) n-gramas a considerar
#     min_topic_size: número mínimo de documentos por tópico
BERTOPIC_PARAMS = {
    "language": "portuguese",        # ← personalize: "english", "spanish"…
    "n_gram_range": (1, 2),          # ← personalize: (1,1) para só uni-gramas, (2,3) para bi-+tri
    "min_topic_size": 5              # ← personalize: 5, 10, 20…
}
print("✅ Configurações salvas! Pode prosseguir para a próxima célula.")

In [ ]:
# 2.n Carregue o DataFrame
df = pd.read_csv(CSV_PATH, sep=',', encoding='utf-8')

# 2.n Selecione a coluna de texto que será trabalhada
#    Certifique-se de definir TEXT_COLUMN antes (ex.: "seu_texto")
texts = df[TEXT_COLUMN].dropna().astype(str).tolist()

# 2.n Pronto: agora 'texts' é uma lista de strings que você pode passar ao seu pipeline
print(f"Carregados {len(texts)} itens da coluna '{TEXT_COLUMN}'.")

In [ ]:
# 2.n Inicializar spaCy
nlp = spacy.load(SPACY_MODEL)

# 2.n Pré-processamento
def preprocess(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        lemma = token.lemma_.lower()
        if token.is_alpha and not token.is_stop and lemma not in CUSTOM_STOP_WORDS:
            tokens.append(lemma)
    return " ".join(tokens)

docs = df[TEXT_COLUMN].dropna().astype(str).map(preprocess).tolist()

# 4.3 Cria um CountVectorizer que já exclui as suas stopwords
vectorizer_model = CountVectorizer(
    ngram_range=BERTOPIC_PARAMS["n_gram_range"], # Use the value from the dictionary
    stop_words=CUSTOM_STOP_WORDS,
    max_df=MAX_DOC_FREQ      # ← filtra termos super-frequentes
)

# 🌟 NUVEM DE PALAVRAS - VERIFICAÇÃO DE STOPWORDS

🔍 OBJETIVO DESTA CÉLULA:
Visualizar as palavras mais frequentes para ajudar a identificar termos irrelevantes que devem ser adicionados como stopwords.

👉 COMO USAR:
1. Execute a célula para gerar a nuvem
2. Observe as palavras destacadas
3. Se aparecerem termos irrelevantes:
   - Volte à célula de CONFIGURÁVEIS
   - Adicione-os em CUSTOM_STOP_WORDS
   - Execute novamente as células subsequentes

📌 EXEMPLOS COMUNS DE STOPWORDS A ADICIONAR:
- Nomes de instituições
- Termos muito genéricos
- Palavras repetitivas do seu contexto específico

In [ ]:
# 5 Núvem de palavras para visualizar o dataset.

# 5.1 Combine todos os textos num único string
text = " ".join(docs)

# 5.2. Monte o conjunto de stopwords (padrão + customizadas)
stopwords = STOPWORDS.union(set(CUSTOM_STOP_WORDS))

# 5.3. Gere a nuvem de palavras
wc = WordCloud(
    stopwords=stopwords,
    width=800,
    height=400,
    background_color="white"   # opcional: defina a cor de fundo
).generate(text)

# 5.4. Exiba o gráfico
plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

# 3️⃣ MODELAGEM DE TÓPICOS

ETAPA PRINCIPAL: Aqui o modelo irá:
1. Analisar todos os textos
2. Identificar padrões
3. Agrupar em tópicos temáticos

⏳ Tempo estimado: Varia conforme quantidade de textos
   (2-10 minutos para conjuntos médios)

🔍 O que será mostrado:
- Tabela com tópicos encontrados e suas frequências
- Palavras-chave de cada tópico
- Amostra dos dados classificados

In [ ]:
# @title ⚙️ Inicialização do BERTopic

# 3.1 Inicialize o BERTopic com TODAS as configurações da célula de parâmetros
topic_model = BERTopic(
    language=BERTOPIC_PARAMS["language"],       # Usa o idioma definido nos parâmetros
    vectorizer_model=vectorizer_model,          # Vectorizer que já criamos com MAX_DOC_FREQ
    min_topic_size=BERTOPIC_PARAMS["min_topic_size"],  # Tamanho mínimo do tópico
    n_gram_range=BERTOPIC_PARAMS["n_gram_range"]       # Faixa de n-gramas
)

# 3.2 Conferência de segurança para garantir que está usando as stopwords corretas
print(f"Configurações atribuídas:")
print(f"- Idioma: {BERTOPIC_PARAMS['language']}")
print(f"- Tamanho mínimo de tópico: {BERTOPIC_PARAMS['min_topic_size']}")
print(f"- N-gram range: {BERTOPIC_PARAMS['n_gram_range']}")
print(f"- Stopwords personalizadas: {CUSTOM_STOP_WORDS[:3]}... (total: {len(CUSTOM_STOP_WORDS)})")
print(f" ")
print(f" Aguarde ...")
print(f" ")

# 3.3 Processamento dos documentos
topics, probs = topic_model.fit_transform(docs)

# 3.4 Armazenamento dos resultados
df_result = df.copy()
df_result["BERtopic"] = topics

print(f"✅ Modelo de classificação aplicado com sucesso!")

In [ ]:
# @title 4️⃣ VISUALIZAÇÕES DOS TÓPICOS

print("\n" + "="*60)
print("📋 TABELA DE TÓPICOS (ORDENADOS POR FREQUÊNCIA)")
print("="*60)

# DataFrame com informações dos tópicos (Topic, Count, Name)
topic_info = topic_model.get_topic_info()

# Melhoria na exibição da tabela
display(topic_info.style
        .background_gradient(subset=['Count'], cmap='Blues')
        .set_caption("Tópicos identificados pelo BERTopic"))

# --------------------------------------------
# 🏷️ MAPEAMENTO DE NOMES PARA TÓPICOS
# --------------------------------------------

print("\n" + "="*60)
print("🔤 PALAVRAS-CHAVE POR TÓPICO")
print("="*60)

# Dicionário de mapeamento (ID tópico → Nome)
topic_map = {
    row['Topic']: row['Name']
    for _, row in topic_info.iterrows()
}

# Exibe palavras-chave de cada tópico formatadas
for topic_id in sorted(topic_info['Topic']):
    palavras_chave = topic_model.get_topic(topic_id)

    # Formatação melhorada
    palavras_formatadas = [
        f"{word} ({score:.2f})"
        for word, score in palavras_chave[:5]  # Top 5 palavras por tópico
    ]

    print(f"\n🔹 Tópico {topic_id} ({topic_map.get(topic_id, 'Outlier')}):")
    print("   📌 Principais termos:", " | ".join(palavras_formatadas))
    print("   📊 Documentos:", topic_info[topic_info['Topic'] == topic_id]['Count'].values[0])

# --------------------------------------------
# 👀 PRÉVIA DOS DADOS CLASSIFICADOS
# --------------------------------------------

print("\n" + "="*60)
print("📄 AMOSTRA DOS RESULTADOS (5 primeiras e últimas linhas)")
print("="*60)

# Configura pandas para mostrar mais colunas
pd.set_option('display.max_columns', None)

# Exibe início e fim do DataFrame
print("\n⬆️ PRIMEIRAS LINHAS:")
display(df_result.head().style.set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6'
}))

print("\n⬇️ ÚLTIMAS LINHAS:")
display(df_result.tail().style.set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6'
}))


# Recupera as estatísticas originais do modelo
raw_topic_info = topic_model.get_topic_info()

# Calcula porcentagens
total_raw_docs = raw_topic_info['Count'].sum()
raw_topic_info['% do Total'] = (raw_topic_info['Count'] / total_raw_docs * 100).round(1)
# EXIBIÇÃO INTERATIVA
# --------------------------------------------------

print("\n" + "="*60)
print(f"📈 DISTRIBUIÇÃO DOS {total_raw_docs} DOCUMENTOS")
print("="*60)
display(raw_topic_info[['Topic', 'Name', 'Count', '% do Total']]
        .rename(columns={
            'Topic': 'ID Tópico',
            'Name': 'Palavras-Chave (Auto)',
            'Count': '📊 Documentos'
        })
        .style
        .format({'% do Total': '{:.1f}%'})
        .bar(subset=['📊 Documentos'], color='#4b8bbe')
        .set_caption("Distribuição dos Tópicos"))


# VERIFICAÇÃO DOS RESULTADOS

✅ SE OS RESULTADOS ACIMA ESTIVEREM SATISFATÓRIOS, PROSSIGA PARA AS PRÓXIMAS CÉLULAS.

⚠️ CASO CONTRÁRIO, VOLTE À SEÇÃO 'CONFIGURAÇÕES' E AJUSTE:
1. CUSTOM_STOP_WORDS: Adicione termos irrelevantes que apareceram
2. MIN_TOPIC_SIZE: Aumente para menos tópicos ou diminua para mais detalhado
3. MAX_DOC_FREQ: Reduza para filtrar palavras muito frequentes

🔄 PROCEDIMENTO:
1. Ajuste os parâmetros
2. Execute novamente desde o pré-processamento
3. Verifique esta visualização novamente

🔎 O QUE ANALISAR:
1. Tópicos com <5 docs podem ser fundidos ou descartados;
2. Outliers (>20%) podem indicar necessidade de ajuste no modelo;
3. Palavras-chave repetidas sugerem stopwords faltantes.

# 📊 VISUALIZAÇÕES INTERATIVAS DOS TÓPICOS

Esta seção gera 3 tipos de visualizações para ajudar na interpretação dos tópicos:

1️⃣ GRÁFICO DE BARRAS - Mostra as palavras mais relevantes de cada tópico
2️⃣ MAPA DE DISTÂNCIA - Ilustra como os tópicos se relacionam entre si
3️⃣ MATRIZ DE SIMILARIDADE - Mostra o grau de similaridade entre tópicos

🔍 Como usar:
- Passe o mouse sobre os elementos para ver detalhes
- Clique e arraste para explorar os gráficos
- Use os menus interativos (quando disponíveis)

In [ ]:
# @title 1️⃣ GRÁFICO DE BARRAS - PALAVRAS-CHAVE POR TÓPICO
# O que você está vendo:
# - Cada barra representa um tópico (exceto outliers -1)
# - As palavras são ordenadas por importância (score)
# - O comprimento da barra mostra a relevância do termo
#
# O que observar:
# - Se as palavras fazem sentido para o tópico
# - Se há termos irrelevantes que deveriam ser stopwords
topic_model.visualize_barchart()

In [ ]:
# @title 2️⃣ MAPA DE DISTÂNCIA - RELAÇÃO ENTRE TÓPICOS
# O que você está vendo:
# - Cada bolha representa um tópico
# - Tópicos próximos são semanticamente similares
# - Tamanho da bolha = quantidade de documentos

# O que observar:
# - Agrupamentos naturais de tópicos
# - Tópicos isolados (potencialmente únicos)
# - Outliers distantes do núcleo
# - Mínimo de 4 tópicos para funcionar
topic_model.visualize_topics()

In [ ]:
# @title 3️⃣ MATRIZ DE SIMILARIDADE - AFINIDADE ENTRE TÓPICOS
# O que você está vendo:
# - Tabela onde cada célula mostra a similaridade (0-1)
# - 1 = tópicos idênticos, 0 = completamente diferentes
# - Diagonal principal sempre = 1 (autossimilaridade)

# O que observar:
# - Tópicos com similaridade > 0.8 (potencial para fusão)
# - Tópicos únicos (baixa similaridade com os demais)
topic_model.visualize_heatmap()

# 🏷️ PERSONALIZAÇÃO DOS NOMES DOS TÓPICOS (OPCIONAL)

📌 ETAPA DE NOMEAÇÃO DOS TÓPICOS

Aqui você pode atribuir nomes descritivos aos tópicos identificados pelo modelo, substituindo os termos de palavras-chave por rótulos significativos.

✏️ COMO USAR:
1. Analise as palavras-chave de cada tópico (geradas anteriormente)
2. Edite o dicionário abaixo com nomes que representem cada grupo
3. Execute a célula para aplicar as mudanças

🖍️ EXEMPLO PRÁTICO:
Se o tópico 0 tem as palavras: "aluno, ensino, personalização",
um bom nome seria "Personalização do Ensino"

⚠️ IMPORTANTE:
- Mantenha o tópico -1 como "Outliers" (dados não classificados)
- Os números devem corresponder aos IDs dos tópicos

In [ ]:
# Dicionário de mapeamento (ID -> Nome do tópico)
labels_map = {
    -1: "Outliers",                       # Documentos não classificados
     0: "Nome do Tópico 1",
     1: "Nome do Tópico 2",
     2: "Nome do Tópico 3",
     3: "Nome do Tópico 4",
     4: "Nome do Tópico 5",
}

# --------------------------------------------
# APLICAÇÃO DOS RÓTULOS (não precisa editar)
# --------------------------------------------
print("✅ Tópicos renomeados com sucesso!")
print("\n📋 Relação de tópicos atualizada:")
for topic_id, topic_name in labels_map.items():
    print(f"Tópico {topic_id}: {topic_name}")

# 📌 RESUMO DOS TÓPICOS RENOMEADOS E VISUALIZAÇÕES

As células abaixo geram resumos e visualizações dos tópicos com:
- Nomes personalizados dos tópicos (definidos anteriormente)
- Contagem de documentos associados a cada tópico
- Dados percentuais implícitos (calculados sobre o total)

▸ O que você pode analisar:
1. Distribuição dos documentos entre tópicos
2. Proporção de outliers (tópico -1)
3. Tópicos dominantes e minoritários

In [ ]:
# Obtém estatísticas brutas do modelo
topic_info = topic_model.get_topic_info()
# Aplica os nomes personalizados dos tópicos
topic_info['Tópico'] = topic_info['Topic'].map(labels_map)
# Formata o quadro final de exibição
quadro = topic_info.loc[:, ['Tópico', 'Count']].rename(
    columns={'Count': 'Quantidade de Documentos'}
)
# Adiciona coluna de porcentagem
total_docs = quadro['Quantidade de Documentos'].sum()
quadro['% do Total'] = (quadro['Quantidade de Documentos'] / total_docs * 100).round(1)

# EXIBIÇÃO FORMATADA (saída interativa)
# --------------------------------------------------
print(f"\n🔍 Distribuição dos {total_docs} documentos analisados")
print(f" ")
print(f" ")
display(quadro.style
        .format({'% do Total': '{:.1f}%'})
        .bar(subset=['Quantidade de Documentos'], color='#5fba7d')
        .set_caption("Frequência de Documentos por Tópico"))


In [ ]:
# Agora vamos fazer um gráfico com a distribuição de itens por tópico

# Obter o DataFrame com frequência de tópicos
topic_info = topic_model.get_topic_info()
# Aplicar os labels
topic_info['Label'] = topic_info['Topic'].map(labels_map)
# Montar a paleta de cores, reservando cinza para o tópico -1
palette = qualitative.Plotly  # ex: ['#636EFA', '#EF553B', '#00CC96', ...]
topic_colors = {
    -1:    'lightgrey',
     0:    palette[0],
     1:    palette[1],
     2:    palette[2],
     3:    palette[3],
     4:    palette[4],
}
# Preparar os dados para o gráfico de pizza
sizes  = topic_info['Count']
labels = topic_info['Label']
colors = [ topic_colors[t] for t in topic_info['Topic'] ]
# Desenhar o gráfico
fig, ax = plt.subplots(figsize=(8, 8))
ax.pie(
    sizes,
    labels=labels,
    colors=colors,
    autopct='%1.1f%%',
    startangle=90,
    textprops={'color':'black'}
)
ax.axis('equal')  # garante círculo
plt.title("Distribuição de Itens por Tópico")
plt.show()

In [ ]:
# Agora vamos visualizar a matrix de similaridade com os nomes dos topicos que voce atribuiu:
topic_model.set_topic_labels(labels_map)
# Gerar o bar chart
fig = topic_model.visualize_heatmap(custom_labels=True)
# Atualizar o título
fig.update_layout(
    title_text="Matriz de Similaridade",              # o texto que você quiser
    # title_x=0.5,                                    # opcional: centraliza horizontalmente
    # title_font_size=18                              # opcional: ajusta o tamanho da fonte
)
# Exibir
fig.show()

In [ ]:
# Agora vamos visualizar o mapa de distancia entre topicos com os nomes dos topicos que voce sugeriu:
from plotly.colors import qualitative
topic_model.set_topic_labels(labels_map)
# Gerar o gráfico normalmente
fig = topic_model.visualize_topics(
    custom_labels=True,
    width=900,
    height=700,
    title="Mapa de Distância entre Tópicos"
)
# Preparar a paleta padrão
palette = qualitative.Plotly
# Mapear tópico → cor
topic_colors = {topic: palette[i] for i, topic in enumerate(labels_map)}
# Recolorir e adicione rótulos internos com texto preto
scatter = fig.data[0]
# Extrair o índice do tópico de cada bolha
topics_in_fig = [int(row[0]) for row in scatter.customdata]
# Aplicar a cor das bolhas
scatter.marker.color = [topic_colors[t] for t in topics_in_fig]
# Exibir o texto dentro das bolhas
scatter.mode = 'markers+text'
scatter.text = [labels_map[t] for t in topics_in_fig]
scatter.textposition = 'middle center'
# Ajustar fonte do texto
scatter.textfont = dict(size=11, color='black')
# Mostrar o gráfico
fig.show()

# 📊 ENTENDENDO OS SCORES DE PALAVRAS-CHAVE

O score (de 0 a 1) representa a importância relativa de cada termo dentro de um tópico, considerando:

* FREQUÊNCIA: Quão comum a palavra é no tópico específico
* DISTINÇÃO: Quão exclusiva ela é desse tópico (vs outros tópicos)

▰ Score 0.9 → Palavra MUITO característica do tópico

▰ Score 0.2 → Palavra pouco representativa

ANÁLISE PRÁTICA:
* Barras mais longas = termos mais definidores do tópico
* Compare scores entre palavras do mesmo tópico

In [ ]:
# Agora vamos plotar o score de palavras por tópico.
import math
# === CONFIGURAÇÃO FÁCIL ===
n_topics = len([k for k in labels_map.keys() if k != -1])  # Exclui outliers (-1)
n_top_words  = 5   # quantas palavras principais exibir por tópico
n_cols       = 3   # quantas colunas por linha
fig_width    = 4   # largura de cada subplot (em polegadas)
fig_height   = 4   # altura de cada subplot (em polegadas)
title        = "Score de Palavras por Tópico"

# Montar a paleta padrão e mapear tópico → cor
palette = qualitative.Plotly
topic_colors = {topic: palette[i] for i, topic in enumerate(labels_map)}
# Calcular layout
n_rows = math.ceil(n_topics / n_cols)
fig, axes = plt.subplots(
    n_rows, n_cols,
    figsize=(fig_width * n_cols, fig_height * n_rows),
    sharex=False
)
fig.suptitle(title, fontsize=16)
axes = axes.flatten()
for idx in range(n_topics):
    ax = axes[idx]
    # Extrair as top n_top_words de cada tópico (já em ordem decrescente)
    topic_kw = topic_model.get_topic(idx)[:n_top_words]
    words, scores = zip(*topic_kw)
    # Plot horizontal e inverte eixo y para maior em cima
    ax.barh(words, scores, color=topic_colors[idx])
    ax.invert_yaxis()
    ax.set_title(labels_map[idx], fontsize=12)
    ax.set_xlabel("Score")
    ax.grid(axis="x", linestyle="--", alpha=0.5)

# Desligar eixos extras se houver
for j in range(n_topics, len(axes)):
    axes[j].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# 5️⃣ EXPORTAÇÃO DOS RESULTADOS


💾 ETAPA FINAL: EXPORTAR RESULTADOS

Aqui você pode salvar os resultados da análise em formatos para:
1. Publicação web (HTML interativo)
2. Compartilhamento (CSV)

Escolha as opções que deseja utilizar:

In [ ]:
# @title 🌐 OPÇÃO PARA WEB:
# Gera um arquivo HTML com a matriz de similaridade entre tópicos, que pode ser embedado em sites ou compartilhado.

# ▸ Como usar:
# 1. Execute esta célula
# 2. Encontre o arquivo 'topic_heatmap.html' no painel esquerdo
# 3. Faça o download ou copie para seu servidor web

heatmap_html = topic_model.visualize_heatmap()
heatmap_html.write_html("topic_heatmap.html")
print("✅ HTML interativo salvo como 'topic_heatmap.html'")


In [ ]:
# @title 📊 EXPORTAÇÃO PARA CSV (ANÁLISE EM PLANILHAS)
# --------------------------------------------------

# Como usar:
# 1. Execute esta célula
# 2. Clique no ícone 📁 no menu esquerdo
# 3. Localize 'resultado_topics.csv'
# 4. Clique em ⋮ → Fazer download
# Garante que a coluna Nome_Topico existe e está correta


if 'Nome_Topico' not in df_result.columns:
    df_result['Nome_Topico'] = df_result['BERtopic'].map(labels_map).fillna("Outliers")
else:
    # Atualiza apenas os valores faltantes
    df_result['Nome_Topico'] = df_result['Nome_Topico'].fillna(
        df_result['BERtopic'].map(labels_map).fillna("Outliers")
    )

# --------------------------------------------------
# 2. EXPORTAÇÃO PARA CSV
# --------------------------------------------------
df_result.to_csv(
    'resultado_topics.csv',
    index=False,
    encoding='utf-8-sig',  # Codificação que preserva acentos
    columns=[col for col in df_result.columns if col != 'Unnamed: 0']  # Remove colunas indesejadas
)
print("\n✅ Dados classificados salvos como 'resultado_topics.csv'")
print("   Encoding: UTF-8 com BOM (compatível com Excel)")

BERTopic: Neural topic modeling with a class-based TF-IDF procedure: https://doi.org/10.48550/arXiv.2203.05794